<a href="https://colab.research.google.com/github/hinsley/colabs/blob/master/Training_Parameterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Prescription Notation

- ```AxB, C @ L | XX:XX```: `A` sets of `B` repetitions, followed by one set of `C` repetitions, all at `L` load, with `XX:XX` (minutes:seconds) rest time between sets.

# Metrics

- `INOL`: Intensity \* Number Of Lifts. Popularized by Hristo Hristov, this metric is frequently used to gauge the systemic stress induced by a training prescription, as well as to compare the relative stimulus incurred from multiple *groups* of sets with varying loading schemes and of varying quantities of repetitions. The name of this metric is a misnomer, as demonstrated by denominator in the formula: $\text{INOL} = \frac{\text{Sets Performed} \cdot \text{Repetitions Per Set}}{100\% - \text{(% Intensity)}}$.
- `Maximum Repetitions`: The quantity of repetitions of a movement a trainee may perform in order to reach failure.
- `REQ`: Repetition-Endurance Quotient. This is used as a linear gauge of how close to failure a set is. The formula is $\text{REQ} = \frac{\text{Repetitions Performed}}{\text{Maximum Repetitions}}$. This metric may be used to predict technical quality of repetitions in a set.
- `RIR`: Repetitions In Reserve. After completing a set, RIR denotes the quantity of *additional* repetitions that would be required to reach failure. For instance, if one is able to complete 10 repetitions, but only performs 6, the set has a RIR rating of 4.
- `RIR falloff`: Given a training prescription with at least the following constraints: `_xB @ L | XX:XX` (quantity of sets is not required), RIR falloff is an integer-valued function (typically linear) with real-valued result ($\text{RIR falloff} : \mathbb{N} \mapsto \mathbb{R}$) expressing how much RIR will decrease (or, perhaps usefully in the context of circuit training, increase) per set performed.